# Set up


In [ ]:
!pip install kaggle
!mkdir .kaggle
import json
token = {"username":"yuzhongfinally","key":"da5ea7094c4adb8f29251a2e3b240da7"}
with open('/content/gdrive/MyDrive/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)
!chmod 600 /content/gdrive/MyDrive/.kaggle/kaggle.json
!cp /content/gdrive/MyDrive/.kaggle/kaggle.json /root/.kaggle/
!kaggle config set -n path -v /content/gdrive/MyDrive/

# Pre-processing


In [ ]:
import torch
import numpy as np
import torch.nn as nn

In [ ]:
class Dataset_train(torch.utils.data.Dataset):
  def __init__(self, X, Y, offset , context ):
    self.X = X
    self.Y = Y

    index_map_X = []

    for i, x in enumerate(X):
      for j, xx in enumerate(x):
        index_map_X.append((i, j))

    self.index_map = index_map_X
        
    self.length = len(self.index_map)
    self.context = context
    self.offset = offset
        
    for i, x in enumerate(self.X):
        self.X[i] = np.pad(x, ((context, context), (0, 0)), 'constant', constant_values=0)
            
        
  def __len__(self):
      
      return self.length
    
  def __getitem__(self, index):
      
      i, j = self.index_map[index]
      
      start_j = j + self.offset - self.context
      
      end_j = j + self.offset + self.context + 1
      
      xx = self.X[i][start_j:end_j,:]

      yy = self.Y[i][j]
     
      return xx ,yy  


In [ ]:
def dataloader(data, label, offset, context, train_loader_args):
  dataset = Dataset_train(data, label, offset, context)
  dataloader = torch.utils.data.DataLoader(dataset,**train_loader_args)
                                        # collate_fn=Dataset_train.collate_fn)
  return dataloader

# Training

In [ ]:
class MLP(torch.nn.Module):
    # you can use the layer sizes as initialization arguments if you want to
    def __init__(self,size_list):
        super(MLP, self).__init__()
        layers = []
        self.size_list = size_list
        for i in range(len(size_list) - 2):
          
          layers.append(nn.Linear(size_list[i], size_list[i+1]))
          layers.append(nn.LeakyReLU())
          layers.append(nn.BatchNorm1d(size_list[i+1]))
          layers.append(nn.Dropout(0.3))
          
          
        layers.append(nn.Linear(size_list[-2], size_list[-1]))
        self.net = nn.Sequential(*layers)

    def forward(self, input_val):
 
      return self.net(input_val)


In [ ]:
from torch.optim.lr_scheduler import StepLR
def train_parameters(model,criterion, iter, lr, data):

  model.train()
  #optimizer = torch.optim.SGD(model.parameters(),lr = 0.0005, momentum = 0.9)
  #optimizer = torch.optim.Adam(model.parameters(), lr = lr, betas= (0.9, 0.99))
  optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=2.5e-6)
  scheduler = StepLR(optimizer, step_size = 5, gamma = 0.5)
  
  model = model.to(device= 'cuda')

  for i in range(iter):
    print('Epoch:',i)

    avg_loss = 0.0
    for batch_num,(X_train, y_train) in enumerate(data):
      optimizer.zero_grad()

      X_train = X_train.to(device='cuda')
      y_train = y_train.to(device='cuda')

    
      X_train = X_train.view(X_train.size(0), -1)
      y_hat = model(X_train.float())

      loss = criterion(y_hat, y_train.long())
      loss.backward()
      optimizer.step()
      avg_loss += loss.item()
      if batch_num % 99 == 1:
          print('loss', avg_loss/100)
          avg_loss = 0.0

      del X_train
      del y_train
      del y_hat
      torch.cuda.empty_cache()
      

    scheduler.step()
  

# Validating

In [ ]:
from sklearn.metrics import accuracy_score

def model_val(net, dataloader_val):
  with torch.no_grad():
    net.eval()

    total_preds = []
    truth = []
    
    for i,  (X_test, y_test) in enumerate(dataloader_val):
      X_test = X_test.to(device='cuda')
      X_test = X_test.view(X_test.size(0), -1)
      y_test = y_test.to(device='cuda')
       
      
      #output
      output = net(X_test.float())
      loss = criterion(output, y_test.long())

      output = output.cpu()
      y_test = y_test.cpu()
      

      output = output.detach().numpy()
      true = y_test.numpy()
      
      output = np.argmax(output,axis = 1)
      
      #reshape
      
      true = true.reshape(-1,1)
      output = output.reshape(-1,1)

      total_preds.append(output)
      truth.append(true)

      del X_test
      del y_test
      del output
      torch.cuda.empty_cache()

      
    
 
  print('finished')
  truth, total_preds = np.vstack(truth), np.vstack(total_preds)
  acc = accuracy_score(truth, total_preds)
  print(acc)
  return acc, loss.item()
   

# run loop to find best parameter

In [ ]:
#tuning
train = np.load('train.npy', allow_pickle = True)
train_labels = np.load('train_labels.npy',allow_pickle = True)
context = 25
offset = 25
batch_size = 64
num_workers = 1

train_loader_args = dict(shuffle = True, batch_size = batch_size, num_workers = num_workers) 
dataloader_train = dataloader(train,train_labels, offset, context, train_loader_args )

del train
del train_labels
#!unzip dev.npy.zip
#!unzip dev_labels.npy.zip
dev = np.load('dev.npy', allow_pickle = True)
dev_labels = np.load('dev_labels.npy',allow_pickle = True)
dev_loader_args = dict(shuffle = True, batch_size = batch_size, num_workers =0)
dataloader_val  = dataloader(dev,dev_labels, offset, context, dev_loader_args)
del dev
del dev_labels



In [ ]:
case_accuracy = model_val(model, dataloader_val)

finished
0.7447583128078817


In [ ]:
# against learning rate
lr = [0.1]
#build the network

context = 25
input = (2 * int(context)+ 1) * 40
output = 71

hidden_lyr =[input,3550,3150, 2800, 2500,2000,2000,1650,1350,1000,650,output]
criterion = nn.CrossEntropyLoss()
epoch =25
report_lr = {}
for l in lr:
  #model = MLP(hidden_lyr)
  model = torch.load('model1_0.72')
  print(model)
  accu_loss = train_parameters(model,criterion, epoch, l, dataloader_train)
  case_accuracy = model_val(model, dataloader_val)
  report_lr[l] = case_accuracy

  if case_accuracy > 0.75:
    torchname = "model_lr_bn_"+str(l)
    torch.save(model, torchname)

  print(report_lr)


# add Test set and submit

In [ ]:

#!unzip test.npy.zip

test = np.load('test.npy', allow_pickle= True)

In [ ]:
class load_test(torch.utils.data.Dataset):
    
    def __init__(self, X, offset, context):
        
        self.X = X
        
        index_map_X = []
        
        for i, x in enumerate(X):
            for j, xx in enumerate(x):
                index_pair_X = (i, j)
                index_map_X.append(index_pair_X)
                
        self.index_map = index_map_X
        
        self.length = len(self.index_map)
        
        self.context = context
        self.offset = offset

        for i, x in enumerate(self.X):
            self.X[i] = np.pad(x, ((context, context), (0, 0)), 'constant', constant_values=0)
        
    def __len__(self):

        return self.length
    
    def __getitem__(self, index):

        i, j = self.index_map[index]
        
        start_j = j + self.offset - self.context

        end_j = j + self.offset + self.context + 1

        xx = self.X[i][start_j:end_j,:]

        return xx


In [ ]:

test_loader_args = dict(shuffle = False, batch_size = batch_size, num_workers = num_workers, pin_memory =  True) 

dataset = load_test(test,offset, context )
dataloader_test = torch.utils.data.DataLoader(dataset,**test_loader_args)



In [ ]:

preds_total = []
model.cuda()
with torch.no_grad():
  for X_test in dataloader_test:
    X_test = X_test.to(device='cuda')
    y_hat = model(X_test.view(X_test.size(0), -1).float())
    
    y_hat = y_hat.cpu()
    y_hat = y_hat.detach().numpy()
    
    preds = np.argmax(y_hat, axis = 1)
    preds = preds.reshape(-1,1)
    preds_total.append(preds)


In [ ]:
preds_final = []
for i in preds_total:
  for t in i:
    preds_final.append(int(t))


In [ ]:
import pandas as pd
df = pd.DataFrame({'id':np.arange(len(preds_final)), "label":preds_final})
df.to_csv(r"propose_prediction.csv", index=False)

In [ ]:
!kaggle competitions submit -c 11785-spring2021-hw1p2 -f propose_prediction.csv -m "Message"

100% 15.5M/15.5M [00:05<00:00, 3.23MB/s]
Successfully submitted to 11785-Spring2021-Hw1P2